In [1]:
import mindspore

In [2]:
mindspore.set_context(device_target='CPU')
mindspore.run_check()

MindSpore version:  2.1.1
The result of multiplication calculation is correct, MindSpore has been installed on platform [CPU] successfully!


In [3]:
import cv2
import numpy as np
import os

In [4]:
# 初始化：读取文件夹中的所有图片
print("[INFO] adding images.........................")
path = "Images"  # 指定图片文件夹路径
images = []  # 存储读取的图片
for filename in os.listdir(path):
    filenames = path + '\\' + filename
    img_orig = cv2.imread(filenames, 1)  # 读取图片，1表示保留图片颜色信息
    if img_orig is None:
        print("Warning: No Pictures")
    else:
        images.append(img_orig)

[INFO] adding images.........................


In [5]:
# 使用OpenCV进行图像拼接
print("[INFO] stitching images.........................")
stitcher = cv2.Stitcher.create(cv2.Stitcher_PANORAMA)  # 创建图像拼接器
status, pano = stitcher.stitch(images)  # 执行拼接

[INFO] stitching images.........................


In [6]:
print("[INFO] cropping images.........................")
if status == cv2.Stitcher_OK:
    # 对拼接后的图像进行处理，提取全景图轮廓并进行裁剪，以去除多余的部分
    stitched = cv2.copyMakeBorder(pano, 10, 10, 10, 10, cv2.BORDER_CONSTANT, (0, 0, 0))  # 添加边界
    gray = cv2.cvtColor(stitched, cv2.COLOR_BGR2GRAY)  # 转换为灰度图
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY)[1]  # 二值化处理
    cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]  # 提取轮廓

    # 计算轮廓的最小外接矩形，并创建掩膜以移除该矩形外的区域
    mask = np.zeros(thresh.shape, dtype="uint8")
    (x, y, w, h) = cv2.boundingRect(cnts[0])
    cv2.rectangle(mask, (x, y), (x + w, y + h), 255, -1)

    # 使用腐蚀操作移除最小外接矩形外部的像素，直到该矩形内所有像素值为0
    minRect = mask.copy()
    sub = mask.copy()
    while cv2.countNonZero(sub) > 0:
        minRect = cv2.erode(minRect, None)
        sub = cv2.subtract(minRect, thresh)

    # 再次提取轮廓并裁剪拼接后的图像，以得到最终的全景图
    cnts = cv2.findContours(minRect, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]
    (x, y, w, h) = cv2.boundingRect(cnts[0])
    stitched = stitched[y:y + h, x:x + w]

    # 显示最终的全景图
    cv2.imshow('stitched', stitched)
    # cv2.imwrite('stitched.jpg', stitched)  # 保存全景图
    cv2.waitKey(0)  # 等待按键退出
    cv2.destroyAllWindows()
else:
    print('图像匹配的特征点不足')

[INFO] cropping images.........................
